# QuantFredom Video 1 - Converting TradingView RSI strategy to Python




In [97]:
# YT Video - https://www.youtube.com/watch?v=-PepEj8BM6g
# import libs
import os
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from quantfreedom.enums import *
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
# from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from numba import njit
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('BYBT_KEY')
api_secret = os.getenv('BYBT_SECRET')

mufex_main = Bybit(
  api_key=api_key,
  secret_key=api_secret,
  use_test_net=True
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
# Testing testing
y = np.arange(10)
print(y)
y = np.roll(y, -1)
y

[0 1 2 3 4 5 6 7 8 9]


array([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [102]:
# Fetch candles // NOTE: Use dl_ex_candles function instead
candles = mufex_main.get_candles(symbol='BTCUSDT', timeframe='5m', candles_to_dl=300)
print(f'First row of candles tohlcv: \n{candles[0]}')
print()
# Get Prices from Close col
prices = candles[:, 4]
print(f'Close price array: \n{prices}')
period = 14

First row of candles tohlcv: 
[1.7106864e+12 6.7423500e+04 6.7450100e+04 6.7266300e+04 6.7417100e+04
 9.0343000e+01]

Close price array: 
[67417.1 67322.  67328.  67660.7 67491.8 67371.9 67410.  67332.7 67434.4
 67440.4 67639.  67734.3 67811.1 68112.8 68113.1 68032.  68088.1 68148.8
 67901.2 68040.  68057.8 68197.3 68138.3 68286.  68178.8 68124.6 67950.9
 68080.6 68106.8 68100.4 68065.2 68182.1 68266.8 68414.5 68200.1 68235.
 68292.  68426.8 68549.  68632.8 68384.7 68551.8 68433.3 68384.1 68339.1
 68326.1 68213.2 68311.5 68305.1 68220.  68201.4 68229.3 68193.3 68164.1
 68283.9 68254.  68259.9 68227.6 68386.1 68280.5 68399.  68492.8 68439.1
 68582.7 68638.7 68583.9 68682.8 68554.6 68579.6 68504.8 68436.2 68443.8
 68500.9 68386.7 68338.8 68321.8 68413.8 68333.8 68354.3 68233.8 68321.1
 68312.  68316.3 68246.3 68222.8 68279.8 68051.3 68194.  68318.6 68532.
 68550.2 68459.1 68603.4 68521.1 68688.4 68925.7 68842.4 68551.4 68483.5
 68282.1 68362.1 68269.  68295.8 68342.8 68191.  68247.4 6829

In [100]:
# Prior price 
prices_shift = np.roll(prices, 1)
prices_shift[0] = np.nan
print(f'Current Price: {prices[-1]}')
print(f'Previous Price: {prices_shift[-1]}') 

# Pct Change
pchgs = (prices - prices_shift) / prices_shift
print(f'Change: {pchgs[-1]}')
alpha = 1 / period
# gain
gains = np.where(pchgs > 0, pchgs, 0)
rma_gains = np.full_like(gains, np.nan)
# loss
losses = np.where(pchgs < 0, abs(pchgs), 0)
rma_losses = np.full_like(losses, np.nan)

rma_gains[period] = gains[1 : period + 1].mean()
rma_losses[period] = losses[1 : period + 1].mean()

# Relative Moving Average
for i in range(period, + 1, gains.size):
  rma_gains[i] = alpha * gains[i] + (1 - alpha) * rma_gains[i - 1]
  rma_losses[i] = alpha * losses[i] + (1 - alpha) * rma_losses[i - 1]

rs = rma_gains / rma_losses

rsi = 100 - (100 / (1 + rs))

print(rsi)

Current Price: 67429.8
Previous Price: 67264.8
Change: 0.002452991757947693
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan 71.50168951         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan

In [101]:
# RSI Indicator Function
def qf_calc_rsi(prices: np.array, period: int):
  prices_shift = np.roll(prices, 1)
  prices_shift[0] = np.nan
  pchg = (prices - prices_shift) / prices_shift

  alpha = 1 / period
  gain = np.where(pchg > 0, pchg, 0)
  avg_gain = np.full_like(gain, np.nan)

  loss = np.where(pchg < 0, abs(pchg), 0)
  avg_loss = np.full_like(loss, np.nan)

  avg_gain[period] = gain[1 : period + 1].mean()
  avg_loss[period] = loss[1 : period + 1].mean()

  for i in range(period, + 1, gain.size):
    avg_gain[i] = alpha * gain[i] + (1 - alpha) * avg_gain[i - 1]
    avg_loss[i] = alpha * loss[i] + (1 - alpha) * avg_loss[i - 1]

  rs = avg_gain / avg_loss

  rsi = 100 - (100 / (1 + rs))
  print(rsi)
  return rsi

